In [ ]:
# Imports
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

plt.rcParams['figure.figsize'] = (15.0, 8.0) # set default size of plots
plt.rcParams['figure.facecolor'] = 'white'

pd.set_option('display.max_rows', None)

matplotlib.rcParams.update({'font.size': 15})

In [ ]:
TIME_RETRIEVAL_REE = 0
TIME_RETRIEVAL_TEE = 10
TIME_RETRIEVAL_REE_WASM = 2
TIME_RETRIEVAL_TEE_WASM = 18

In [ ]:
def compute_mean_std(name, time):
    return pd.DataFrame(
        {
            "Name": [name],
            "Mean": [time.median()],
            "Std": [time.std()]
        }
    )

def empty_export_row(name):
    return pd.DataFrame(
        {
            "Name": [name],
            "Mean": [0],
            "Std": [0]
        }
    )

def formatForLatex(n, round_digits = 2):
    roundedAsString = str(round(n, round_digits))
    if "." in roundedAsString:
        roundedAsString = roundedAsString.rstrip("0")
        roundedAsString = roundedAsString.rstrip(".")
    return f"{roundedAsString}"

def number_to_text(text):
    return text.replace("0", "Zero").replace("1", "One").replace("2", "Two")

In [ ]:
#
## Attester data
#

In [ ]:
# Import the dataset
attester_columns = ["QuoteStart","QuoteEnd","Message0MemStart","Message0KeygenStart","Message0KeygenEnd", \
                   "Message1MemStart","Message1AsymCryptoStart","Message1KeygenStart","Message1SymCryptoStart", \
                   "Message1SymCryptoEnd","Message2MemStart","Message2SymCryptoStart","Message2SymCryptoEnd"]

attester = pd.read_csv("../logs/messages012-time/messages012-attester.csv", names = attester_columns)

attester

In [ ]:
# Summary of the gathered timespans
attester_spans = pd.DataFrame()
attester_spans["Quote"] = attester["QuoteEnd"] - attester["QuoteStart"] \
                        - 2*TIME_RETRIEVAL_TEE/2

attester_spans["Message0Mem"] = attester["Message0KeygenStart"] - attester["Message0MemStart"] \
                        - 2*TIME_RETRIEVAL_TEE/2

attester_spans["Message0Keygen"] = attester["Message0KeygenEnd"] - attester["Message0KeygenStart"] \
                        - 2*TIME_RETRIEVAL_TEE/2

attester_spans["Message1Mem"] = attester["Message1AsymCryptoStart"] - attester["Message1MemStart"] \
                        - 2*TIME_RETRIEVAL_TEE/2

attester_spans["Message1AsymCrypto"] = attester["Message1KeygenStart"] - attester["Message1AsymCryptoStart"] \
                        - 2*TIME_RETRIEVAL_TEE/2

attester_spans["Message1Keygen"] = attester["Message1SymCryptoStart"] - attester["Message1KeygenStart"] \
                        - 2*TIME_RETRIEVAL_TEE/2

attester_spans["Message1SymCrypto"] = attester["Message1SymCryptoEnd"] - attester["Message1SymCryptoStart"] \
                        - 2*TIME_RETRIEVAL_TEE/2

attester_spans["Message2Mem"] = attester["Message2SymCryptoStart"] - attester["Message2MemStart"] \
                        - 2*TIME_RETRIEVAL_TEE/2

attester_spans["Message2SymCrypto"] = attester["Message2SymCryptoEnd"] - attester["Message2SymCryptoStart"] \
                        - 2*TIME_RETRIEVAL_TEE/2

# We assign the execution time of the quote generation to the message2 (simpler summary for the paper)
attester_spans["Message2AsymCrypto"] = attester_spans["Quote"]

attester_spans

In [ ]:
attester_mem_export = pd.DataFrame()
attester_mem_export = attester_mem_export.append(compute_mean_std("Message0", attester_spans["Message0Mem"]))
attester_mem_export = attester_mem_export.append(compute_mean_std("Message1", attester_spans["Message1Mem"]))
attester_mem_export = attester_mem_export.append(compute_mean_std("Message2", attester_spans["Message2Mem"]))
attester_mem_export.set_index("Name", inplace=True)

attester_mem_export

In [ ]:
attester_keygen_export = pd.DataFrame()
attester_keygen_export = attester_keygen_export.append(compute_mean_std("Message0", attester_spans["Message0Keygen"]))
attester_keygen_export = attester_keygen_export.append(compute_mean_std("Message1", attester_spans["Message1Keygen"]))
attester_keygen_export = attester_keygen_export.append(empty_export_row("Message2"))
attester_keygen_export.set_index("Name", inplace=True)

attester_keygen_export

In [ ]:
attester_asym_crypto_export = pd.DataFrame()
attester_asym_crypto_export = attester_asym_crypto_export.append(empty_export_row("Message0"))
attester_asym_crypto_export = attester_asym_crypto_export.append(compute_mean_std("Message1", attester_spans["Message1AsymCrypto"]))
attester_asym_crypto_export = attester_asym_crypto_export.append(compute_mean_std("Message2", attester_spans["Message2AsymCrypto"]))
attester_asym_crypto_export.set_index("Name", inplace=True)

attester_asym_crypto_export

In [ ]:
attester_sym_crypto_export = pd.DataFrame()
attester_sym_crypto_export = attester_sym_crypto_export.append(empty_export_row("Message0"))
attester_sym_crypto_export = attester_sym_crypto_export.append(compute_mean_std("Message1", attester_spans["Message1SymCrypto"]))
attester_sym_crypto_export = attester_sym_crypto_export.append(compute_mean_std("Message2", attester_spans["Message2SymCrypto"]))
attester_sym_crypto_export.set_index("Name", inplace=True)

attester_sym_crypto_export

In [ ]:
attester_sym_crypto_export["Mean"].values

In [ ]:
index = ['msg0', 'msg1', 'msg2']
df = pd.DataFrame(
    {
        'mem': attester_mem_export["Mean"].values,
        'keygen': attester_keygen_export["Mean"].values,
        'asym': attester_asym_crypto_export["Mean"].values,
        'sym': attester_sym_crypto_export["Mean"].values,
    }, index=index)
ax = df.plot.bar(rot=0)

In [ ]:
#
## Verifier data
#

In [ ]:
# Import the dataset
verifier_columns = ["Message0Keygen1Start","Message0MemStart","Message0Keygen2Start","Message0Keygen2End", \
                   "Message1Mem1Start","Message1AsymCryptoStart","Message1SymCryptoStart","Message1Mem2Start", \
                   "Message1Mem2End","Message2Mem1Start","Message2SymCryptoStart", "Message2Mem2Start", \
                   "Message2AsymCryptoStart","Message2AsymCryptoEnd"]

verifier = pd.read_csv("../logs/messages012-time/messages012-verifier.csv", names = verifier_columns)

verifier

In [ ]:
# Summary of the gathered timespans
verifier_spans = pd.DataFrame()

verifier_spans["Message0Keygen"] = verifier["Message0MemStart"] - verifier["Message0Keygen1Start"] \
                        + verifier["Message0Keygen2End"] - verifier["Message0Keygen2Start"] \
                        - 4*TIME_RETRIEVAL_TEE/2

verifier_spans["Message0Mem"] = verifier["Message0Keygen2Start"] - verifier["Message0MemStart"] \
                        - 2*TIME_RETRIEVAL_TEE/2

verifier_spans["Message1Mem"] = verifier["Message1AsymCryptoStart"] - verifier["Message1Mem1Start"] \
                        + verifier["Message1Mem2End"] - verifier["Message1Mem2Start"] \
                        - 4*TIME_RETRIEVAL_TEE/2

verifier_spans["Message1AsymCrypto"] = verifier["Message1SymCryptoStart"] - verifier["Message1AsymCryptoStart"] \
                        - 2*TIME_RETRIEVAL_TEE/2

verifier_spans["Message1SymCrypto"] = verifier["Message1Mem2Start"] - verifier["Message1SymCryptoStart"] \
                        - 2*TIME_RETRIEVAL_TEE/2

verifier_spans["Message2Mem"] = verifier["Message2SymCryptoStart"] - verifier["Message2Mem1Start"] \
                        + verifier["Message2AsymCryptoStart"] - verifier["Message2Mem2Start"] \
                        - 4*TIME_RETRIEVAL_TEE/2

verifier_spans["Message2SymCrypto"] = verifier["Message2Mem2Start"] - verifier["Message2SymCryptoStart"] \
                        - 2*TIME_RETRIEVAL_TEE/2

verifier_spans["Message2AsymCrypto"] = verifier["Message2AsymCryptoEnd"] - verifier["Message2AsymCryptoStart"] \
                        - 2*TIME_RETRIEVAL_TEE/2

verifier_spans

In [ ]:
# Compute the mean and std
verifier_mem_export = pd.DataFrame()
verifier_mem_export = verifier_mem_export.append(compute_mean_std("Message0", verifier_spans["Message0Mem"]))
verifier_mem_export = verifier_mem_export.append(compute_mean_std("Message1", verifier_spans["Message1Mem"]))
verifier_mem_export = verifier_mem_export.append(compute_mean_std("Message2", verifier_spans["Message2Mem"]))
verifier_mem_export.set_index("Name", inplace=True)

verifier_mem_export

In [ ]:
verifier_keygen_export = pd.DataFrame()
verifier_keygen_export = verifier_keygen_export.append(compute_mean_std("Message0", verifier_spans["Message0Keygen"]))
verifier_keygen_export = verifier_keygen_export.append(empty_export_row("Message1"))
verifier_keygen_export = verifier_keygen_export.append(empty_export_row("Message2"))
verifier_keygen_export.set_index("Name", inplace=True)

verifier_keygen_export

In [ ]:
verifier_sym_crypto_export = pd.DataFrame()
verifier_sym_crypto_export = verifier_sym_crypto_export.append(empty_export_row("Message0"))
verifier_sym_crypto_export = verifier_sym_crypto_export.append(compute_mean_std("Message1", verifier_spans["Message1SymCrypto"]))
verifier_sym_crypto_export = verifier_sym_crypto_export.append(compute_mean_std("Message2", verifier_spans["Message2SymCrypto"]))
verifier_sym_crypto_export.set_index("Name", inplace=True)

verifier_sym_crypto_export

In [ ]:
verifier_asym_crypto_export = pd.DataFrame()
verifier_asym_crypto_export = verifier_asym_crypto_export.append(empty_export_row("Message0"))
verifier_asym_crypto_export = verifier_asym_crypto_export.append(compute_mean_std("Message1", verifier_spans["Message1AsymCrypto"]))
verifier_asym_crypto_export = verifier_asym_crypto_export.append(compute_mean_std("Message2", verifier_spans["Message2AsymCrypto"]))
verifier_asym_crypto_export.set_index("Name", inplace=True)

verifier_asym_crypto_export

In [ ]:
index = ['msg0', 'msg1', 'msg2']
df = pd.DataFrame(
    {
        'mem': verifier_mem_export["Mean"].values,
        'keygen': verifier_keygen_export["Mean"].values,
        'asym': verifier_asym_crypto_export["Mean"].values,
        'sym': verifier_sym_crypto_export["Mean"].values,
    }, index=index)
ax = df.plot.bar(rot=0)

In [ ]:
# Export CSV
#
# File format: message_id, mean, stddev

attester_mem_export.to_csv('../logs/messages012-time/attester_mem_formatted.csv')
attester_keygen_export.to_csv('../logs/messages012-time/attester_keygen_formatted.csv')
attester_sym_crypto_export.to_csv('../logs/messages012-time/attester_sym_crypto_formatted.csv')
attester_asym_crypto_export.to_csv('../logs/messages012-time/attester_asym_crypto_formatted.csv')

verifier_mem_export.to_csv('../logs/messages012-time/verifier_mem_formatted.csv')
verifier_keygen_export.to_csv('../logs/messages012-time/verifier_keygen_formatted.csv')
verifier_sym_crypto_export.to_csv('../logs/messages012-time/verifier_sym_crypto_formatted.csv')
verifier_asym_crypto_export.to_csv('../logs/messages012-time/verifier_asym_crypto_formatted.csv')

In [ ]:
# Compute ratio for the paper

message1_sym_crypto_time = verifier_sym_crypto_export.loc[verifier_sym_crypto_export.index == "Message1"]["Mean"].values[0]
message1_asym_crypto_time = verifier_asym_crypto_export.loc[verifier_asym_crypto_export.index == "Message1"]["Mean"].values[0]
message1_crypto_ratio = message1_asym_crypto_time / message1_sym_crypto_time

In [ ]:
# Export LaTeX

def export_to_latex(file, prefix, dataset):
    for _, row in dataset.iterrows():
        name = number_to_text(row.name)
        mean = int(row["Mean"])
        std = int(row["Std"])
        file.write(f"\\def\\bm{prefix}{name}MeanInMicrosec{{{mean:,}}}\n")
        file.write(f"\\def\\bm{prefix}{name}MeanInMs{{{formatForLatex(mean/1000,0)}}}\n")
        #file.write(f"\\def\\bm{prefix}{name}Stddev{{{std}}}\n")

def write(file, name, number, digits_round):
    f.write(f"\\def\\bm{name}{{{formatForLatex(number, digits_round)}}}\n")
        
f = open("../logs/messages012-time/export.tex", "w")
export_to_latex(f, "AttesterMem", attester_mem_export)
export_to_latex(f, "AttesterKeygen", attester_keygen_export)
export_to_latex(f, "AttesterSymCrypto", attester_sym_crypto_export)
export_to_latex(f, "AttesterAsymCrypto", attester_asym_crypto_export)
export_to_latex(f, "VerifierMem", verifier_mem_export)
export_to_latex(f, "VerifierKeygen", verifier_keygen_export)
export_to_latex(f, "VerifierSymCrypto", verifier_sym_crypto_export)
export_to_latex(f, "VerifierAsymCrypto", verifier_asym_crypto_export)
write(f, number_to_text("Message1AsymVsSymRatio"), message1_crypto_ratio ,0)
f.close()